In [1]:
from models.rnn.rnn_plain import RNN_Plain
from models.rnn.rnn_attention import RNN_Attention

from lang import load_data_int_seq
from utils import accuracy_score
%load_ext autoreload
%autoreload 2

In [3]:
output_lang, input_lang, train, X_test, y_test = load_data_int_seq()

## Plain RNN

In [26]:
algo = RNN_Plain(symbols = "+*-0123456789t", output_sequence_length = 9, encoded_seq_length = 9, num_epochs= 2, input_size=input_lang.n_words, hidden_size=256, output_size=output_lang.n_words, embedding_size = 28, calc_magnitude_on=False)

In [48]:
algo.train(input_lang, output_lang, train)

original pairs
[('-1,-4,-7,-10,-13,-16,-19,-22', '1+1+0-t*3'), ('-5,-4,-3,-2,-1,0,1,2', '0*7*1-6+t'), ('4,3,2,1,0,-1,-2,-3', '5-t-t+0+t')]
input data
['54,59,64,69,74,79,84,89', '5,-3,-11,-19,-27,-35,-43,-51', '-1,-2,-3,-4,-5,-6,-7,-8']
target data
['t*6+7*7-t', '9+t+4-9*t', '0-8*3*0-t']


KeyError: '54,59,64,69,74,79,84,89'

In [ ]:
pred = algo.infer(input_lang, output_lang, X_test[:1000])
pred[:25]

In [ ]:
accuracy_score(pred, y_test[:1000])

### Plain RNN with Magnitude loss

In [ ]:
magn_algo = RNN_Plain(symbols = "+*-0123456789t", output_sequence_length = 9, encoded_seq_length = 9, num_epochs= 5000, input_size=input_lang.n_words, hidden_size=512, output_size=output_lang.n_words, calc_magnitude_on=True)
magn_algo.train(input_lang, output_lang, train)

In [ ]:
pred = magn_algo.infer(input_lang, output_lang, X_test[:1000])
pred[:25]

In [ ]:
accuracy_score(pred, y_test[:1000])

##  Attention-based

In [ ]:
algo_attn = RNN_Attention(symbols = "+*-0123456789t", output_sequence_length = 9, encoded_seq_length = 9, num_epochs= 5000, input_size=input_lang.n_words, hidden_size=512, output_size=output_lang.n_words, calc_magnitude_on=False)
algo_attn.train(input_lang, output_lang, train)

In [ ]:
pred = algo_attn.infer(input_lang, output_lang, X_test[:1000])
pred[:25]

In [ ]:
accuracy_score(pred, y_test[:1000])

### Attention with magnitude loss

In [ ]:
algo_attn_magn = RNN_Attention(symbols = "+*-0123456789t", output_sequence_length = 9, encoded_seq_length = 9, num_epochs= 5000, input_size=input_lang.n_words, hidden_size=512, output_size=output_lang.n_words, calc_magnitude_on=True)
algo_attn_magn.train(input_lang, output_lang, train)

In [ ]:
pred = algo_attn_magn.infer(input_lang, output_lang, X_test[:1000])
pred[:25]

In [ ]:
accuracy_score(pred, y_test[:1000])